In [1]:
import argparse
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from dataclasses import dataclass
from joblib import Parallel, delayed
from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

from model import GModel
from myutils import roc_auc, translate_result, filter_target
from load_data import load_data
from optimizer import Optimizer
from sampler import NewSampler

In [3]:
@dataclass
class Args:
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
    data: str = "nci"
    n_jobs: int = 1 # ← ここをコア数に応じて調整
    lr: float = 5e-4
    epochs: int = 1000

In [4]:
args = Args()
target_option = "cell"  # "cell" か "drug"

In [5]:
# データ読み込み
res, drug_feature, exprs, mut, cna, null_mask = load_data(args)

load nci
response matrix (res) shape: (977, 59)
exprs shape: (60, 23059)
mut shape: (60, 9307)
cna shape: (60, 23232)
59
exprs shape: (59, 23059)
mut shape: (59, 9307)
cna shape: (59, 23232)
drug_feature shape: (976, 920)
response matrix (res) shape: (59, 976)
null_mask shape: (59, 976)


In [6]:
def run_single_model(exprs, cna, mut, drug_feature, res_mat, null_mask, target_dim, target_index, args, seed):
    sampler = NewSampler(res_mat, null_mask, target_dim, target_index)
    val_labels = sampler.test_data[sampler.test_mask]

    model = GModel(
        adj_mat=sampler.train_data.float(),
        gene=exprs,
        cna=cna,
        mutation=mut,
        sigma=2,
        k=11,
        iterates=3,
        feature_drug=drug_feature,
        n_hid1=192,
        n_hid2=36,
        alpha=5.74,
        device=args.device,
    )
    opt = Optimizer(
        model=model,
        train_data=sampler.train_data,
        test_data=sampler.test_data,
        test_mask=sampler.test_mask,
        train_mask=sampler.train_mask,
        evaluate_fun=roc_auc,
        lr=args.lr,
        epochs=args.epochs,
        device=args.device,
    ).to(args.device)
    _, true_data, predict_data = opt()
    return true_data, predict_data

In [7]:
# ターゲット次元と統計
target_dim = 0 if target_option == "cell" else 1
samples = res.shape[target_dim]
cell_sum = np.sum(res.values, axis=1)
drug_sum = np.sum(res.values, axis=0)

In [8]:
# 並列実行用ラッパー
def process_target(seed, target_index):
    try:
        return run_single_model(
            exprs=exprs,
            cna=cna,
            mut=mut,
            drug_feature=drug_feature,
            res_mat=res.values,
            null_mask=null_mask,
            target_dim=target_dim,
            target_index=target_index,
            args=args,
            seed=seed,
        )
    except Exception as e:
        print(f"❌ Failed at target {target_index}: {e}")
        return None

In [9]:
# 結果格納用
true_data_s = pd.DataFrame()
predict_data_s = pd.DataFrame()
skipped_targets = []
passed_targets = []

# スキップチェック
for target_index in range(samples):
    label_vec = res.iloc[target_index] if target_dim == 0 else res.iloc[:, target_index]
    passed, reason, pos, neg, total = filter_target(label_vec)

    if passed:
        passed_targets.append(target_index)
    else:
        skipped_targets.append((target_index, reason, pos, neg, total))

# スキップ情報表示
print(f"\n🚫 Skipped Targets: {len(skipped_targets)}")
for idx, reason, pos, neg, total in skipped_targets:
    print(f"Target {idx}: skipped because {reason} (total={total}, pos={pos}, neg={neg})")

# 並列実行（max_njobs を args.n_jobs に変更可能）
results = Parallel(n_jobs=args.n_jobs)(
    delayed(process_target)(seed, target_index)
    for seed, target_index in enumerate(tqdm(passed_targets, desc=f"MOFGCN ({args.data} - {target_option})"))
)

# 結果の統合（None を除外）
for r in results:
    if r is not None:
        true_data, pred_data = r
        true_data_s = pd.concat([true_data_s, translate_result(true_data)], ignore_index=True)
        predict_data_s = pd.concat([predict_data_s, translate_result(pred_data)], ignore_index=True)


🚫 Skipped Targets: 2
Target 15: skipped because low_positive_ratio (total=488, pos=7, neg=481)
Target 33: skipped because low_negative_ratio (total=427, pos=419, neg=8)


MOFGCN (nci - cell):   0%|          | 0/57 [00:00<?, ?it/s]

epoch:   0 loss:0.927380 auc:0.5257
epoch:  20 loss:0.249320 auc:0.4684
epoch:  40 loss:0.150524 auc:0.3832
epoch:  60 loss:0.115222 auc:0.4099
epoch:  80 loss:0.100356 auc:0.4163


MOFGCN (nci - cell):   2%|▏         | 1/57 [00:02<01:52,  2.02s/it]

epoch: 100 loss:0.092887 auc:0.4228
epoch: 120 loss:0.088448 auc:0.4219
epoch: 140 loss:0.085507 auc:0.4219
epoch: 160 loss:0.083440 auc:0.4228
Fit finished.
epoch:   0 loss:0.759236 auc:0.5284
epoch:  20 loss:0.247624 auc:0.7207
epoch:  40 loss:0.150348 auc:0.7864
epoch:  60 loss:0.114585 auc:0.8174
epoch:  80 loss:0.099909 auc:0.8198
epoch: 100 loss:0.092486 auc:0.8139
epoch: 120 loss:0.088092 auc:0.8107
epoch: 140 loss:0.085216 auc:0.8083
epoch: 160 loss:0.083207 auc:0.8084
epoch: 180 loss:0.081728 auc:0.8081


...

epoch: 200 loss:0.081820 auc:0.9503
epoch: 220 loss:0.080893 auc:0.9487
Fit finished.
epoch:   0 loss:0.796835 auc:0.4306
epoch:  20 loss:0.251083 auc:0.4888
epoch:  40 loss:0.154996 auc:0.4872
epoch:  60 loss:0.118543 auc:0.5068
epoch:  80 loss:0.102958 auc:0.5054
epoch: 100 loss:0.094833 auc:0.5054
epoch: 120 loss:0.089915 auc:0.5079
epoch: 140 loss:0.086642 auc:0.5099
epoch: 160 loss:0.084345 auc:0.5087
epoch: 180 loss:0.082672 auc:0.5072
epoch: 200 loss:0.081416 auc:0.5070


MOFGCN (nci - cell):  98%|█████████▊| 56/57 [00:42<00:00,  1.64it/s]

epoch: 220 loss:0.080445 auc:0.5066
Fit finished.
epoch:   0 loss:0.807118 auc:0.5321
epoch:  20 loss:0.227804 auc:0.7406
epoch:  40 loss:0.136899 auc:0.8217
epoch:  60 loss:0.107220 auc:0.8445
epoch:  80 loss:0.095309 auc:0.8411
epoch: 100 loss:0.089171 auc:0.8358
epoch: 120 loss:0.085452 auc:0.8331
epoch: 140 loss:0.082972 auc:0.8284


MOFGCN (nci - cell): 100%|██████████| 57/57 [00:43<00:00,  1.32it/s]

epoch: 160 loss:0.081230 auc:0.8270
epoch: 180 loss:0.079966 auc:0.8257
epoch: 200 loss:0.079031 auc:0.8262
epoch: 220 loss:0.078324 auc:0.8261
Fit finished.


In [11]:
# Save
true_path = f"mofgcn_true_{args.data}_{target_option}.csv"
pred_path = f"mofgcn_pred_{args.data}_{target_option}.csv"
true_data_s.to_csv(true_path, index=False)
predict_data_s.to_csv(pred_path, index=False)

print(f"\n✅ Done. Results saved to:\n  - {true_path}\n  - {pred_path}")


✅ Done. Results saved to:
  - mofgcn_true_nci_cell.csv
  - mofgcn_pred_nci_cell.csv
